In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train=pd.read_csv('/kaggle/input/bengaliai-cv19/train.csv')
test=pd.read_csv('/kaggle/input/bengaliai-cv19/test.csv')
submission=pd.read_csv('/kaggle/input/bengaliai-cv19/sample_submission.csv')
class_map=pd.read_csv('/kaggle/input/bengaliai-cv19/class_map.csv')

In [ ]:
import keras
# create class which allow to data augmentation for multioutput. 
class MultiOutputDataGenerator(keras.preprocessing.image.ImageDataGenerator):

    def flow(self,
             x,
             y=None,
             batch_size=32,
             shuffle=True,
             sample_weight=None,
             seed=None,
             save_to_dir=None,
             save_prefix='',
             save_format='png',
             subset=None):

        targets = None
        target_lengths = {}
        ordered_outputs = []
        for output, target in y.items():
            if targets is None:
                targets = target
            else:
                targets = np.concatenate((targets, target), axis=1)
            target_lengths[output] = target.shape[1]
            ordered_outputs.append(output)


        for flowx, flowy in super().flow(x, targets, batch_size=batch_size,
                                         shuffle=shuffle):
            target_dict = {}
            i = 0
            for output in ordered_outputs:
                target_length = target_lengths[output]
                target_dict[output] = flowy[:, i: i + target_length]
                i += target_length

            yield flowx, target_dict

In [ ]:
from PIL import Image , ImageOps
from PIL import ImageFilter
def blur (img):
    """ """
    image= Image.fromarray(img).convert('L')
    
    image=image.filter(ImageFilter.GaussianBlur(radius=2))
    
    return image
def contrast(img):
    image=Image.fromarray(img).convert('L')
    image=ImageOps.autocontrast(image)
    return image

In [ ]:
from tqdm.auto import tqdm
import cv2
size=96
def resize(df, size=96, need_progress_bar=True):
    resized = {}
    resize_size=96
    if need_progress_bar:
        for i in tqdm(range(df.shape[0])):
            image=df.loc[df.index[i]].values.reshape(137,236)
            
            img=blur(image)
            img=np.array(img)
            image=contrast(image)
            image=np.array(image)
            _, thresh = cv2.threshold(img, 30, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            contours, _ = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[-2:]

            idx = 0 
            ls_xmin = []
            ls_ymin = []
            ls_xmax = []
            ls_ymax = []
            for cnt in contours:
                idx += 1
                x,y,w,h = cv2.boundingRect(cnt)
                ls_xmin.append(x)
                ls_ymin.append(y)
                ls_xmax.append(x + w)
                ls_ymax.append(y + h)
            xmin = min(ls_xmin)
            ymin = min(ls_ymin)
            xmax = max(ls_xmax)
            ymax = max(ls_ymax)

            roi = image[ymin:ymax,xmin:xmax]
            resized_roi = cv2.resize(roi, (resize_size, resize_size),interpolation=cv2.INTER_AREA)
            resized[df.index[i]] = resized_roi.reshape(-1)
    else:
        for i in range(df.shape[0]):
            #image = cv2.resize(df.loc[df.index[i]].values.reshape(137,236),(size,size),None,fx=0.5,fy=0.5,interpolation=cv2.INTER_AREA)
            image=df.loc[df.index[i]].values.reshape(137,236)
            _, thresh = cv2.threshold(image, 30, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            contours, _ = cv2.findContours(thresh,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[-2:]

            idx = 0 
            ls_xmin = []
            ls_ymin = []
            ls_xmax = []
            ls_ymax = []
            for cnt in contours:
                idx += 1
                x,y,w,h = cv2.boundingRect(cnt)
                ls_xmin.append(x)
                ls_ymin.append(y)
                w=max(w,resize_sizee)
                h=max(h,resize_size)
                ls_xmax.append(x + w)
                ls_ymax.append(y + h)
            xmin = min(ls_xmin)
            ymin = min(ls_ymin)
            xmax = max(ls_xmax)
            ymax = max(ls_ymax)

            roi = image[ymin:ymax,xmin:xmax]
            resized_roi = cv2.resize(roi, (resize_size, resize_size),interpolation=cv2.INTER_AREA)
            resized[df.index[i]] = resized_roi.reshape(-1)
    resized = pd.DataFrame(resized).T
    return resized

# CNN architecture 

In [ ]:
from keras.layers import Dense,Dropout,Input,MaxPooling2D, BatchNormalization,Conv2D,Flatten
from keras.models import Model
Inputs=Input(shape=(size,size,1))
model=Conv2D(64,(3,3),padding='same',input_shape=(size,size,1),activation='relu')(Inputs)
model=Conv2D(64,(3,3),padding='same',activation='relu')(model)
model=MaxPooling2D((2,2),strides=(2,2))(model)
model=BatchNormalization(momentum=0.15)(model)
model=Conv2D(128,(3,3),padding='same',activation='relu')(model)
model=Conv2D(128,(3,3),padding='same',activation='relu')(model)
model=MaxPooling2D((2,2),strides=(2,2))(model)
model=BatchNormalization(momentum=0.15)(model)
model=Conv2D(256,(3,3),padding='same',activation='relu')(model)
model=Conv2D(256,(3,3),padding='same',activation='relu')(model)
model=Conv2D(256,(3,3),padding='same',activation='relu')(model)
model=MaxPooling2D((2,2),strides=(2,2))(model)
model=BatchNormalization(momentum=0.15)(model)
model=Conv2D(512,(3,3),padding='same',activation='relu')(model)
model=Conv2D(512,(3,3),padding='same',activation='relu')(model)
model=Conv2D(512,(3,3),padding='same',activation='relu')(model)
model=MaxPooling2D((2,2),strides=(2,2))(model)
model=BatchNormalization(momentum=0.15)(model)
model=Conv2D(512,(3,3),padding='same',activation='relu')(model)
model=Conv2D(512,(3,3),padding='same',activation='relu')(model)
model=Conv2D(512,(3,3),padding='same',activation='relu')(model)
model=MaxPooling2D((2,2),strides=(2,2))(model)
model=BatchNormalization(momentum=0.15)(model)
model=Flatten()(model)
model=Dense(1024,activation='relu')(model)
dense=Dense(512,activation='relu')(model)
consonant=Dense(7,activation='softmax')(dense)
grapheme=Dense(168,activation='softmax')(dense)
vowel=Dense(11,activation='softmax')(dense)
model=Model(inputs=Inputs,outputs=[consonant,grapheme,vowel])

In [ ]:
from keras.utils import plot_model
plot_model(model,to_file='model.png',show_shapes=True)

In [ ]:
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from keras.callbacks import ReduceLROnPlateau
#Reducing learning rate when accuracy has stopped improving.
#We will use a callback wich will control the accuracy and reduce learning rate by 40% when the quantity
#controlled had not enhanced after 4 epochs 
lr_consonant=ReduceLROnPlateau(monitor='dense_3_accuracy',factor=0.4,patience=4,min_lr=0.00001)
lr_grapheme=ReduceLROnPlateau(monitor='dense_4_accuracy',factor=0.4,patience=4,min_lr=0.00001)
lr_vowel=ReduceLROnPlateau(monitor='dense_5_accuracy',factor=0.4,patience=4,min_lr=0.00001)

In [ ]:
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
for i in range(4):
    df=pd.merge(pd.read_parquet('/kaggle/input/bengaliai-cv19/train_image_data_{}.parquet'.format(i)),train,\
                on='image_id',how='left').drop(['image_id','grapheme'],axis=1)
   
    X=df.drop(['grapheme_root','vowel_diacritic','consonant_diacritic'],axis=1)
    X=resize(X)/255
                                                                                                   
    X=X.to_numpy().reshape(-1,size,size,1)
    Y_consonant=pd.get_dummies(df['consonant_diacritic']).to_numpy()
    Y_grapheme=pd.get_dummies(df['grapheme_root']).to_numpy()
    Y_vowel=pd.get_dummies(df['vowel_diacritic']).to_numpy()
    del(df)
    # Divide the data into training and test data
    xtr,xts,ytr_consonant,yts_consonant,ytr_grapheme,yts_grapheme,ytr_vowel,yts_vowel=train_test_split(X,\
                                                            Y_consonant,Y_grapheme,Y_vowel,test_size=0.1)
    
    del(X)
    del(Y_consonant)
    del(Y_grapheme)
    del(Y_vowel)
    #data augmentation .
    datagen=MultiOutputDataGenerator(rotation_range=10 # rotate the image\
                                    ,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.15)
    datagen.fit(xtr)
    fichier=[]
    #train the model.
    md=model.fit_generator(datagen.flow(xtr,{'dense_3':ytr_consonant,'dense_4':ytr_grapheme,'dense_5':\
                                             ytr_vowel},batch_size=256),steps_per_epoch=len(xtr)/256,\
                          epochs=24,validation_data=(xts,[yts_consonant,yts_grapheme,yts_vowel]),\
                          callbacks=[lr_consonant,lr_grapheme,lr_vowel])
    fichier.append(md)
    # reduce memory
    del(xtr)
    del(xts)
    del(ytr_consonant)
    del(yts_consonant)
    del(ytr_grapheme)
    del(yts_grapheme)
    del(ytr_vowel)
    del(yts_vowel)

In [ ]:
ant=[]
for i in range(4):
    ts=pd.read_parquet('/kaggle/input/bengaliai-cv19/test_image_data_{}.parquet'.format(i))
    ts=ts.drop(['image_id'],axis=1)
    ts=resize(ts,need_progress_bar=True)
    tss=ts.to_numpy()/255
    tss=tss.reshape(-1,size,size,1)
    prediction=model.predict(tss)
    l=len(ts)
    del(ts)
    m=0
    del(tss)
    for j in range(l):
        ant.append(np.argmax(prediction[0][m]))
        ant.append(np.argmax(prediction[1][m]))
        ant.append(np.argmax(prediction[2][m]))
        m+=1

In [ ]:
submission['target']=ant

In [ ]:
submission.to_csv('submission.csv',index=False)

In [ ]:
submission